In [1]:
# Insérez ici le numéro de votre équipe
VOTRE_NUMERO_EQUIPE = 4

In [2]:
# Import des paquets requis et configuration du ROS Master
import sys
sys.path.append('/opt/ros/noetic/lib/python3/dist-packages/')
sys.path.append('/usr/lib/python3/dist-packages')
from os import environ
environ['ROS_MASTER_URI'] = "http://localhost:1137{}/".format(VOTRE_NUMERO_EQUIPE)
import numpy as np
import rospy
from scipy.spatial.transform import Rotation as R
from math import sin, cos
from jackal_msgs.msg import Feedback
from jackal_msgs.msg import Drive
from sensor_msgs.msg import Imu
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Point, Pose, Quaternion, Twist, Vector3
from filterpy.kalman import KalmanFilter
#from lab_utils.project2 import publish_odom_tf

In [3]:
# Créer et démarrer un nouveau noeud
rospy.init_node('dingo_state_estimation', anonymous=True)

In [4]:
# Fonction pour le calcul de l'orientation à partir d'un quaternion
# Entrée : Quaternion [x, y ,z ,w]
# Sortie : Angle de lacet (yaw) en radians
def get_heading_from_quaternion(q):
    r = R.from_quat([q.x, q.y, q.z, q.w])
    angles = r.as_euler('xyz', degrees=False)
    return angles[2]

In [5]:
# Ground truth subscriber callback
ground_truth_msg = Odometry()
def ground_truth_callback(msg):
    global ground_truth_msg
    ground_truth_msg = msg

In [6]:
# UMI subscriber callback
# Dans cette fonction la variable msg contient les mesures de l'UMI
def imu_callback(msg):
    pass
    # Utiliser les valeurs du IMU dans cette fonction

### Configuration du script

Ajoutez ici les données géométriques du robot

In [7]:
# À compléter selon les tâches précédentes
interwheel_distance = 0.46
left_wheel_radius = 0.045
right_wheel_radius = 0.045

prev_left_pos = 0.0
prev_right_pos = 0.0
x_robot_prec = 0.0
y_robot_prec = 0.0
theta_robot_prec = 0.0
x_robot = 0.0
y_robot = 0.0
theta_robot = 0.0
vlin_robot = 0.0
vang_robot = 0.0
# R = 0.0

### Estimer les déplacements du robot

Dans la prochaine cellule ajoutez votre code pour estimer les variables x, y et theta du robot (et leurs vitesses).

In [8]:
# Encodeurs subscriber callback
# Estimation de la position du robot selon les mesures des encodeurs

last_time_ = rospy.Time.now()
def encoders_callback(msg):
    # Définition des variables globales
    global prev_left_pos, prev_right_pos, last_time_
    global theta_robot_prec, theta_robot, x_robot_prec, y_robot_prec, x_robot, y_robot, vlin_robot, vang_robot # Pour que les modifications appliquées à ces variables soient effectives à l'extérieur de la fonction il faut déclarer ses variables comme globales
    
    #Initialisation de l'intervalle de temps dt
    time = rospy.Time.now()
    dt = (time - last_time_).to_sec()
    if dt < 0.0001:
        return
    
    # Récupération de la mesures de chaque encodeur
    left_pos  = msg.drivers[0].measured_travel
    right_pos = msg.drivers[1].measured_travel

    
    # Distance parcourue par chaque roue
    dist_r = right_wheel_radius*(right_pos-prev_right_pos)
    dist_l = left_wheel_radius*(left_pos-prev_left_pos)
    
    # Vitesse de chaque roue 
    vr = dist_r/dt
    vl = dist_l/dt

    # Vitesse lineaire du robot 
    vlin_robot = (vr+vl)/2
    
    # Rayon de la trajectoire (pas utilisé)
    R=(interwheel_distance/2)*((vr+vl)/(vr-vl))
    
    # Vitesse de rotation du robot
    vang_robot = (vr-vl)/interwheel_distance
    
    # Position du robot
    theta_robot = theta_robot_prec + vang_robot * dt
    x_robot = x_robot_prec + dt * vlin_robot * cos(theta_robot)
    y_robot = y_robot_prec + dt * vlin_robot * sin(theta_robot)
        
    # Enregistrez les valeurs pour la prochaine fois
    prev_left_pos    = left_pos
    prev_right_pos   = right_pos
    theta_robot_prec = theta_robot
    x_robot_prec     = x_robot
    y_robot_prec     = y_robot
    last_time_       = rospy.Time.now() # A mettre à jour à chaque itération pour que dt corresponde bien au temps entre deux itérations (Jean)

### Configuration du script

Ajoutez ici les noms des messages ROS

In [9]:
# ROS subscribers et publishers
feedback_sub = rospy.Subscriber('/mobile_manip/dingo_velocity_controller/feedback', Feedback, encoders_callback)
imu_sub = rospy.Subscriber('/imu/data', Imu, imu_callback)
ground_truth_sub = rospy.Subscriber('/mobile_manip/t265/odom/sample', Odometry, ground_truth_callback)
cmd_drive_pub = rospy.Publisher('/mobile_manip/dingo_velocity_controller/cmd_drive', Drive, queue_size=1)

### Tester l'estimation

La prochaine cellule déplace le robot dans une courbe pour 20 secondes et enregistre sa pose réelle la pose estimée dans un rosbag.

Changez les 0 pour les valeurs correspondantes de votre filtre Kalman.

In [10]:
# Création d'un nouveau contenant ROSBAG (en écriture)
import rosbag
test_bag = rosbag.Bag('project2_1.bag', 'w')

# Publie l'odométrie et la TF pendant 20 secondes
start = float(rospy.Time().now().secs)
rate = rospy.Rate(30) # 50hz
while (float(rospy.Time().now().secs) - start) < 20:
    
    # ***TÂCHE***    
    # Debug
    #print('Position x : ',x_robot)
    #print('Position y : ',y_robot)
    #print('Vitesse ang :',vang_robot)
    #print('Position t : ',theta_robot)
    #print('Position t prev : ',theta_robot_prec,'\n')
    
    # ***TÂCHE***
    
    # Déplacer le robot
    cmd_drive_msg = Drive()
    cmd_drive_msg.drivers[0] = 5.0
    cmd_drive_msg.drivers[1] = 7.0
    cmd_drive_pub.publish(cmd_drive_msg)
    
    # Enregistrer le déplacement du robot
    pose = (x_robot,y_robot,0)
    orientation = R.from_euler('xyz', [0, 0, theta_robot], degrees=False).as_quat()
    odometry_msg = Odometry()
    odometry_msg.header.frame_id = "odom"
    odometry_msg.header.stamp = rospy.Time.now()
    odometry_msg.child_frame_id = "base_link"
    odometry_msg.pose.pose = Pose(Point(*pose), Quaternion(*orientation))
    odometry_msg.twist.twist = Twist(Vector3(vlin_robot, 0, 0), Vector3(0, 0, vang_robot))
    
    test_bag.write('/filter', odometry_msg, rospy.Time().now())
    test_bag.write('/ground_truth', ground_truth_msg, rospy.Time().now())
    rate.sleep()
    
# fermer proprement le ROSBAG
test_bag.close()

Position x :  0.00013024517497498337
Position y :  -2.174440081353666e-06
Vitesse ang : 0.0
Position t :  -0.016772139412553375
Position t prev :  -0.016772139412553375 

Position x :  0.008485839072778587
Position y :  -0.0001161854961771255
Vitesse ang : 0.19565111269121585
Position t :  -0.010713572249464329
Position t prev :  -0.010713572249464329 

Position x :  0.017110435542318475
Position y :  -0.0001807156360547323
Vitesse ang : 0.19565257041350653
Position t :  -0.004449534027472742
Position t prev :  -0.004449534027472742 

Position x :  0.02599876965254293
Position y :  -0.00019069286730114465
Vitesse ang : 0.19564965496892514
Position t :  0.0019944847925850028
Position t prev :  0.0019944847925850028 

Position x :  0.03515520407488583
Position y :  -0.00014110883641672468
Vitesse ang : 0.1891831988873688
Position t :  0.008634019157160932
Position t prev :  0.008634019157160932 

Position x :  0.04404788771784585
Position y :  -3.488187656461954e-05
Vitesse ang : 0.18815

Position x :  0.4334746065205012
Position y :  0.06151036288879113
Vitesse ang : 0.19190622412640115
Position t :  0.29841613769531433
Position t prev :  0.29841613769531433 

Position x :  0.441956492468741
Position y :  0.06406929795086405
Vitesse ang : 0.1860286878502886
Position t :  0.3046453932057275
Position t prev :  0.3046453932057275 

Position x :  0.45066264038284465
Position y :  0.06683909063145566
Vitesse ang : 0.19871670266856323
Position t :  0.31119567414988836
Position t prev :  0.31119567414988836 

Position x :  0.4590888803159637
Position y :  0.0695795315988759
Vitesse ang : 0.19554469896399462
Position t :  0.3174782939579197
Position t prev :  0.3174782939579197 

Position x :  0.46751493126305566
Position y :  0.07237854349420102
Vitesse ang : 0.19545140473738964
Position t :  0.3237818116727104
Position t prev :  0.3237818116727104 

Position x :  0.4759435116125457
Position y :  0.07523798056780126
Vitesse ang : 0.19339893175208056
Position t :  0.3301726527

Position x :  0.826718601269826
Position y :  0.2544880005321342
Vitesse ang : 0.1809907996136209
Position t :  0.6139637076336385
Position t prev :  0.6139637076336385 

Position x :  0.8339751595828379
Position y :  0.25948211924812736
Vitesse ang : 0.1934922259786854
Position t :  0.620148741680641
Position t prev :  0.620148741680641 

Position x :  0.841412887311877
Position y :  0.26483186037189405
Vitesse ang : 0.19573128741720452
Position t :  0.6267377397288434
Position t prev :  0.6267377397288434 

Position x :  0.8486049209769464
Position y :  0.2700761611814771
Vitesse ang : 0.19629105277683423
Position t :  0.633157315461531
Position t prev :  0.633157315461531 

Position x :  0.8557668599385216
Position y :  0.27536950510874486
Vitesse ang : 0.19554469896399462
Position t :  0.6396188735961909
Position t prev :  0.6396188735961909 

Position x :  0.8631104298462662
Position y :  0.28087200476753277
Vitesse ang : 0.19591787587041445
Position t :  0.6462660872417945
Positi

Position x :  1.1406697862557034
Position y :  0.5598858554084344
Vitesse ang : 0.17949809198794162
Position t :  0.9297118601591768
Position t prev :  0.9297118601591768 

Position x :  1.1461282367721835
Position y :  0.5672523470857193
Vitesse ang : 0.1946117566979449
Position t :  0.9362984325574872
Position t prev :  0.9362984325574872 

Position x :  1.1513756610027288
Position y :  0.5744315268357737
Vitesse ang : 0.19274587216584585
Position t :  0.9427092386328739
Position t prev :  0.9427092386328739 

Position x :  1.1565780425381043
Position y :  0.5816454242846123
Vitesse ang : 0.17856514972189186
Position t :  0.9490860856097765
Position t prev :  0.9490860856097765 

Position x :  1.161886146518962
Position y :  0.5891069211849056
Vitesse ang : 0.18043103425399123
Position t :  0.9556209730065475
Position t prev :  0.9556209730065475 

Position x :  1.1669962014382704
Position y :  0.5963891323409266
Vitesse ang : 0.20394117935844078
Position t :  0.9620407353276883
Posi

Position x :  1.3441674794810716
Position y :  0.9484671737695888
Vitesse ang : 0.19479834515115493
Position t :  1.246003358260478
Position t prev :  1.246003358260478 

Position x :  1.3469833103681716
Position y :  0.9566614190787289
Vitesse ang : 0.19591787587041434
Position t :  1.2522682521654551
Position t prev :  1.2522682521654551 

Position x :  1.3497434254030136
Position y :  0.9651258884757971
Vitesse ang : 0.19591787587041445
Position t :  1.2586992097937888
Position t prev :  1.2586992097937888 

Position x :  1.3525256329237736
Position y :  0.9738499036229207
Vitesse ang : 0.19554469896399462
Position t :  1.265258913454837
Position t prev :  1.265258913454837 

Position x :  1.3552478940141308
Position y :  0.9825827415441366
Vitesse ang : 0.1981569373089334
Position t :  1.271782045779057
Position t prev :  1.271782045779057 

Position x :  1.357754417815612
Position y :  0.9908054695823448
Vitesse ang : 0.19293246061905575
Position t :  1.277873039245642
Position t 

...

Position x :  1.1551475474768662
Position y :  2.2052103578235425
Vitesse ang : 0.1951715220575748
Position t :  2.19376634514865
Position t prev :  2.19376634514865 

Position x :  1.1499341749717122
Position y :  2.212416739719688
Vitesse ang : 0.19591787587041426
Position t :  2.200198422307704
Position t prev :  2.200198422307704 

Position x :  1.144670569936585
Position y :  2.2195946038089667
Vitesse ang : 0.19629105277683417
Position t :  2.20667080257265
Position t prev :  2.20667080257265 

Position x :  1.1390362727700174
Position y :  2.227171336982775
Vitesse ang : 0.19591787587041434
Position t :  2.2135477066041553
Position t prev :  2.2135477066041553 

Position x :  1.1338439277349468
Position y :  2.234058971712603
Vitesse ang : 0.1910665760869564
Position t :  2.219778641410648
Position t prev :  2.219778641410648 

Position x :  1.1284486762208024
Position y :  2.2411226545112406
Vitesse ang : 0.19629105277683417
Position t :  2.2261726545252465
Position t prev :  2

Position x :  0.8466719605965592
Position y :  2.51630702090727
Vitesse ang : 0.19554469896399462
Position t :  2.5096415644108583
Position t prev :  2.5096415644108583 

Position x :  0.8394826958616896
Position y :  2.521534346614136
Vitesse ang : 0.17539314601732336
Position t :  2.5159743765128018
Position t prev :  2.5159743765128018 

Position x :  0.8318250698349402
Position y :  2.5270261566164987
Vitesse ang : 0.19554469896399443
Position t :  2.5227042488431777
Position t prev :  2.5227042488431777 

Position x :  0.8247932163831982
Position y :  2.5320008255358943
Vitesse ang : 0.18136397652004083
Position t :  2.528858682383853
Position t prev :  2.528858682383853 

Position x :  0.8174986442658612
Position y :  2.537093069900111
Vitesse ang : 0.19479834515115488
Position t :  2.535260905390393
Position t prev :  2.535260905390393 

Position x :  0.809730594455905
Position y :  2.5424399666358184
Vitesse ang : 0.19629105277683417
Position t :  2.542049739671982
Position t p

Position x :  0.45684290976729247
Position y :  2.7162527513343417
Vitesse ang : 0.19106657608695657
Position t :  2.8255940312926393
Position t prev :  2.8255940312926393 

Position x :  0.4481252032001268
Position y :  2.7190703312863698
Vitesse ang : 0.19554469896399462
Position t :  2.832183589106201
Position t prev :  2.832183589106201 

Position x :  0.43965890186308487
Position y :  2.7217464106269733
Vitesse ang : 0.19629105277683417
Position t :  2.8385074449623904
Position t prev :  2.8385074449623904 

Position x :  0.4311798343207077
Position y :  2.7243678471237573
Vitesse ang : 0.1813639765200407
Position t :  2.8448059247889437
Position t prev :  2.8448059247889437 

Position x :  0.4224088632184984
Position y :  2.727017934261129
Vitesse ang : 0.19479834515115493
Position t :  2.8513529404351736
Position t prev :  2.8513529404351736 

Position x :  0.41388327937126074
Position y :  2.7295340798239884
Vitesse ang : 0.19479834515115488
Position t :  2.857667840045609
Posi

Position x :  0.03290316760177215
Position y :  2.7852114582440795
Vitesse ang : 0.19629105277683417
Position t :  3.1348763341490806
Position t prev :  3.1348763341490806 

Position x :  0.024022848772873636
Position y :  2.785242021308549
Vitesse ang : 0.19479834515115493
Position t :  3.141227805096347
Position t prev :  3.141227805096347 

Position x :  0.015133416793987917
Position y :  2.7852161911159894
Vitesse ang : 0.19629105277683417
Position t :  3.147581515105052
Position t prev :  3.147581515105052 

Position x :  0.0062475924109266796
Position y :  2.7851336716495085
Vitesse ang : 0.19629105277683417
Position t :  3.153998665187848
Position t prev :  3.153998665187848 

Position x :  -0.0029126391961758254
Position y :  2.7849886770917167
Vitesse ang : 0.19479834515115493
Position t :  3.160647931306437
Position t prev :  3.160647931306437 

Position x :  -0.011807943139272928
Position y :  2.784789356080505
Vitesse ang : 0.19629105277683417
Position t :  3.16715016572389

Position x :  -0.40014229937547074
Position y :  2.7191230417242886
Vitesse ang : 0.19554469896399443
Position t :  3.4509034364121516
Position t prev :  3.4509034364121516 

Position x :  -0.40857484521438475
Position y :  2.716398294544185
Vitesse ang : 0.19629105277683417
Position t :  3.4571474324103697
Position t prev :  3.4571474324103697 

Position x :  -0.41701474955562173
Position y :  2.7136125869466436
Vitesse ang : 0.19405199133831522
Position t :  3.4634802445123163
Position t prev :  3.4634802445123163 

Position x :  -0.4256999888311113
Position y :  2.7106835435762884
Vitesse ang : 0.17464679220448368
Position t :  3.47002054297423
Position t prev :  3.47002054297423 

Position x :  -0.4341061949470427
Position y :  2.7077882390390093
Vitesse ang : 0.1925592837126358
Position t :  3.476350369660923
Position t prev :  3.476350369660923 

Position x :  -0.44249295087345314
Position y :  2.704840172062912
Vitesse ang : 0.1821103303328805
Position t :  3.4826757182247374
Po

In [11]:
# Arreter le dingo
cmd_drive_msg = Drive()
cmd_drive_msg.drivers[0] = 0.0
cmd_drive_msg.drivers[1] = 0.0
cmd_drive_pub.publish(cmd_drive_msg)